In [375]:
import pandas as pd
import numpy as np

## Load Data

In [376]:
df = pd.read_csv(r'C:\Users\Germán\Github\Proyecto2\data\vehiculos-de-segunda-mano-sample.csv')

In [377]:
df.head(1)

,vehicle_type,make,model,version,fuel,year,kms,power,doors,shift,color,photos,description,price,currency,location,publish_date,update_date,dealer_name,dealer_description,dealer_address,dealer_zip_code,dealer_city,dealer_country_code,dealer_is_professional,dealer_website,dealer_registered_at,date
0,car,Opel,Crossland,1.2 GAS 110 GS Line 5p S/S,Gasolina,2022,5.0,110.0,NaN,manual,NaN,10,"OFERTA EXCLUSIVA DE INTERNET \nNuevo Opel Crossland GSLINE 1.2 Turbo Gasolina 110cv manual de 6 velocidades.\nVehículo dotado del siguiente equipamiento:\nNUEVO A ESTRENAR, MATRICULADO EN FECHA 31.08.2022 \nFAROS ECOLED\nNAVEGADOR 5.0 INTELILINK\nPILOTOS TRASEROS LED\nCÁMARA DELANTERA (Alerta de Cambio Involuntario de Carril)\nPANTALLA INTELILLINK R4.0 7” CON APPLE CAR PLAY & ANDROID AUTO\nFRENOS DE DISCO DELANTEROS Y TRASEROS \nLLANTAS SPORT 17” COLO NEGRO DIAMANTE\nSENSOR APARCAMIENTO TRASERO\nKIT ANTIPINCHAZOS\nANCLAJES ISOFIX PLAZAS TRASERAS\nASIENTO AGR (ASIENTOS ERGONÓMICO)\nELEVALUNAS TRASEROS Y DELANTEROS ELÉCTRICOS\nCLIMATIZADOR DIGITAL BIZONA\nCONS",22900,EUR,Pineda de Mar,2023-02-04,2023-02-04,Sergio Y.,NaN,"2, Carrer de Jacint Benavente, Poblenou, Pineda de Mar, Maresme, Barcelona, Catalunya, 08397, España",8397.0,Pineda de Mar,ES,True,www.cocheauto.com,2017-01-18,2023-02-06


## Duplicados

In [378]:
# Eliminamos duplicados 
df = df.drop_duplicates()

In [379]:
print(df.shape)

(99763, 28)


In [380]:
#Mostrar nombres de cada columna en una lista
print(df.columns)


Index(['vehicle_type', 'make', 'model', 'version', 'fuel', 'year', 'kms',
       'power', 'doors', 'shift', 'color', 'photos', 'description', 'price',
       'currency', 'location', 'publish_date', 'update_date', 'dealer_name',
       'dealer_description', 'dealer_address', 'dealer_zip_code',
       'dealer_city', 'dealer_country_code', 'dealer_is_professional',
       'dealer_website', 'dealer_registered_at', 'date'],
      dtype='object')


Drop columns

In [381]:
#Drop 'vehicle_type','doors','color','photos', 'description','currency','update_date','dealer_description','dealer_address', 'dealer_zip_code','dealer_is_professional','dealer_city', 'dealer_country_code','dealer_website','dealer_registered_at', 'date'
df = df.drop(['dealer_name','vehicle_type','doors','color','photos', 'description','currency','update_date','dealer_description','dealer_address','dealer_is_professional','dealer_city', 'dealer_country_code','dealer_website','dealer_registered_at', 'date'], axis=1)

## Nulls

In [391]:
#Mostramos los nulos en porcetaje con relacion al total por cada fila 
df.isnull().sum()/df.shape[0]*100

make               0.000000
model              0.000000
version            0.000000
fuel               0.000000
year               0.000000
kms                0.000000
power              0.000000
shift              0.000000
price              0.000000
location           0.000000
publish_date       0.000000
dealer_zip_code    0.093121
dtype: float64

### Drop nulls from version and kms

In [383]:
#Drop nuls de "verion", kms, power,dealer_name
df = df.dropna(subset=['version', 'kms'])

### Fixing Fuel, Power and Shift

In [384]:
#Lista de variables a rellenar
variables = ['fuel', 'power', 'shift']
 
# Función para calcular la moda o mediana por modelo
def calculate_stat_by_model(df, var):
    if var == 'power':
        return df.groupby('model')[var].transform(lambda x: x.median() if not pd.isna(x.median()) else None)
    else:
        return df.groupby('model')[var].transform(lambda x: x.mode()[0] if not x.mode().empty else None)
 
# Función para rellenar los valores nulos
def fill_missing_values(df, var):
    stat_by_model = calculate_stat_by_model(df, var)
    df[var] = df[var].fillna(stat_by_model)
    return df
 
# Aplicar la función para cada variable
for var in variables:
    df = fill_missing_values(df, var)

c:\Users\Germán\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [389]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
 
# Mostrar el resultado completo para 'shift'
shift_nulls = df[df['shift'].isnull()][['make', 'model']].value_counts()
print("Shift Nulls:")
print(shift_nulls)

# Mostrar el resultado completo para 'power'
power_nulls = df[df['power'].isnull()][['make', 'model']].value_counts()
print("\nPower Nulls:")
print(power_nulls)

# Mostrar el resultado completo para 'fuel'
fuel_nulls = df[df['fuel'].isnull()][['make', 'model']].value_counts()
print("\nFuel Nulls:")
print(fuel_nulls)

Shift Nulls:
Series([], Name: count, dtype: int64)

Power Nulls:
Series([], Name: count, dtype: int64)

Fuel Nulls:
Series([], Name: count, dtype: int64)


In [386]:
#Rellenamos citroen e-jumpy con 100
df.loc[(df['make'] == 'Citroen') & (df['model'] == 'ë-Jumpy'), 'power'] = 100

In [387]:
#Rellenamos Lotus Esprit con manual
df.loc[(df['make'] == 'Lotus') & (df['model'] == 'Esprit'), 'shift'] = "manual"

In [388]:
modelos = "LaFerrari","Indigo","Esprit"
df.loc[df['model'].isin(modelos), 'fuel'] = 'Gasolina'

# Zip code

In [394]:
#Mostramos la cantidad de filas donde dealer_zip_code es nulo
print(df['dealer_zip_code'].isnull().sum())

0


In [393]:
#Drop dealer_zip_code nulls
df = df.dropna(subset=['dealer_zip_code'])

In [367]:
# df[df['dealer_zip_code']=='Parque Empresarial San Fernando de Henares'].fillna('28830')
# df[df['dealer_zip_code']=='Collado Villalba'].fillna('28400')
# df[df['dealer_zip_code']=='Elx/Elche'].fillna('03203')

,make,model,version,fuel,year,kms,power,shift,price,location,publish_date,dealer_zip_code


# Dtypes

In [397]:
df.dtypes

make                       object
model                      object
version                    object
fuel                       object
year                        int64
kms                       float64
power                     float64
shift                      object
price                       int64
location                   object
publish_date       datetime64[ns]
dealer_zip_code           float64
dtype: object

In [396]:
#Publish date to datetime
df['publish_date'] = pd.to_datetime(df['publish_date'])

# Getting location latitud and longitude